# 使用 qianfan sdk 构建本地评估模型



千帆大模型平台提供了在线进行自动评估的方式，而有时我们希望有更灵活的自动评估方式。

本文将以评估文本摘要效果为例子来介绍

1. 如何使用千帆 sdk 封装自定义本地评估方法
2. 使用函数封装手搓其原理实现。

以下是本文封装部分使用的库与作用
    Dataset：提供数据集接口，用于构造待评估数据集，方便在本地、平台、huggingface多端拉取数据集。
    Prompt：提供数据模板，用于指导模型进行评估，可联网获取prompt或者使用prompt优化功能。
    Service：提供模型服务接口，在evaluateManager中为没有生成答案的数据集提供生成能力。
    EvaluateManager：提供评估任务管理功能，封装了异步调用算子，可以方便地实现多线程评估。

此处使用0.3.0版本以上的qianfan sdk。

In [ ]:
%pip install -U "qianfan>=0.3.0"

进行鉴权认证。

In [128]:
import os

os.environ['QIANFAN_ACCESS_KEY'] = 'your_access_key'
os.environ['QIANFAN_SECRET_KEY'] = 'your_secret_key'

# 使用 Service 对象进行评估时，请按实际情况填写 QPS LIMIT，
# 取值为所有 Service QPS Limit中的最小值
os.environ["QIANFAN_QPS_LIMIT"] = "1"
os.environ['QIANFAN_LLM_API_RETRY_COUNT'] = "3"

## 数据与模板准备

首先构造待评估数据集，此处用本地数据集，也可以拉取千帆平台上的数据集

In [129]:
from qianfan.dataset import Dataset

ds = Dataset.load(data_file="data_summerize/excerpt.jsonl", organize_data_as_group=False, input_columns=["prompt"], reference_column="response")

print(ds[0])

[{'prompt': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'response': [['修改后的立法法全文公布']]}]


然后创建prompt模版，用于引导模型进行评估。

由于摘要任务没有标准答案，所以此处模板不提供，如果是评估问答任务，则需要提供标准答案以评估正确率。

接着指定评估指标和评估步骤。

此处从四个维度进行评估，分别是相关性、连贯性、一致性、流畅度

In [172]:
from qianfan.common import Prompt

evaluation_prompt_template = """
你是一名裁判员，负责为给定新闻的摘要评分。

评价标准：

{criteria}

请你遵照以下的评分步骤：
{steps}


例子：

新闻：

{prompt}

摘要：

{response}


根据答案的综合水平给出0到{max_score}之间的整数评分。
如果答案存在明显的不合理之处，则应给出一个较低的评分。
如果答案符合以上要求并且与参考答案含义相似，则应给出一个较高的评分

你的回答模版如下:
评分: 此处只能回答整数评分
原因: 此处只能回答评分原因
"""

evaluation_prompt = Prompt(evaluation_prompt_template)

相关性

In [131]:
relevance_metric = """
相关性 - 摘要中重要内容的选择。 \ 
摘要只应包含来自源文档的重要信息。 \
惩罚包含冗余和多余信息的摘要。
"""

relevance_steps = """
1. 仔细阅读摘要和源文档。
2. 将摘要与源文档进行比较，并识别文章的主要观点。
3. 评估摘要是否涵盖了文章的主要观点，以及它包含多少无关或冗余信息。
"""

relevance_max_score = 10

连贯性

In [1]:

coherence_metric = """
连贯性 - 所有句子的总体质量。 \
我们将此维度与 DUC 质量问题结构性和连贯性相关， \
其中“摘要应具有良好的结构和组织，不应只是相关信息的堆叠，而应基于从句子到主题有关的信息的连贯性主体进行构建。” \

"""

coherence_steps = """
1. 仔细阅读文章并识别主要主题和关键点。
2. 阅读摘要并与文章进行比较。检查摘要是否涵盖了文章的主要主题和关键点，并以清晰且逻辑的顺序呈现它们
"""

coherence_max_score = 10

一致性

In [133]:
consistency_metric = """
一致性 - 摘要与摘要源的客观对齐。 \
客观一致的摘要只包含与源文档支持的陈述。 \
惩罚包含幻觉事实的摘要。
"""

consistency_steps = """
1. 仔细阅读文章并识别主要事实和细节。
2. 阅读摘要并与文章进行比较。检查摘要是否包含任何不支持的文章的事实错误。
3. 基于评估标准，为一致性分配分数。
"""

consistency_max_score = 10

流畅度

In [134]:
fluency_metric = """
流畅度：摘要的语法、拼写、标点、单词选择和句子结构的质量。
1：差。摘要有很多错误，使它难以理解或听起来不自然。
2：中。摘要有一些影响文本清晰度或流畅性的错误，但要点仍然可理解。
3：好。摘要很少或没有错误，易于阅读和遵循。
"""

fluency_steps = """
读取摘要并基于给定的标准评估其流畅度。从 1 到 3 分配一个流畅度分数。
"""

fluency_max_score = 3

In [135]:
evaluation_metrics = {
    "Relevance": (relevance_metric, relevance_steps, relevance_max_score),
    "Coherence": (coherence_metric, coherence_steps, coherence_max_score),
    "Consistency": (consistency_metric, consistency_steps, consistency_max_score),
    "Fluency": (fluency_metric, fluency_steps, fluency_max_score),
}

通过千帆sdk，我们有两种方法实现

## 方法一：封装本地评估器

使用千帆平台提供的本地评估工具类进行评估
此方法封装了异步请求，可以根据数据量并发进行评估

首先继承LocalEvaluator类，并实现evaluate方法

In [147]:
import json
from typing import Any, Dict, List, Union, Optional

from qianfan import ChatCompletion
from qianfan.common import Prompt
from qianfan.utils.pydantic import Field
from qianfan.evaluation.evaluator import LocalEvaluator
from qianfan.evaluation.consts import (
    QianfanRefereeEvaluatorPromptTemplate,
    QianfanRefereeEvaluatorDefaultMaxScore,
    QianfanRefereeEvaluatorDefaultMetrics,
    QianfanRefereeEvaluatorDefaultSteps,
)


class LocalJudgeEvaluator(LocalEvaluator):

    model: Optional[ChatCompletion] = Field(default=None, description="model object")
    metric_name: str = Field(default="", description="metric name for evaluation")
    evaluation_prompt: Prompt = Field(default=Prompt(QianfanRefereeEvaluatorPromptTemplate), description="concrete evaluation prompt string")
    prompt_metrics: str = Field(default=QianfanRefereeEvaluatorDefaultMetrics, description="evaluation metrics")
    prompt_steps: str = Field(default=QianfanRefereeEvaluatorDefaultSteps, description="evaluation steps")
    prompt_max_score: int = Field(default=QianfanRefereeEvaluatorDefaultMaxScore, description="max score for evaluation")
    
    class Config:
        arbitrary_types_allowed = True
    def evaluate(
        self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
    ) -> Dict[str, Any]:
        """
        使用模型进行本地评估
        :param input: 给定的prompt，
            evaluateManager.eval()的is_chat参数为true时,
            input为对话记录，否则为单字符串prompt
        :param reference: 用户给定的标准答案
        :param output: 大模型生成的结果，eval中由service生成，eval_only中由用户给定

        :return: 评估结果
        """
        if isinstance(input, list):
            if not isinstance(self.model, ChatCompletion):
                raise ValueError(f"model is not an instance of ChatCompletion")
            if len(input)!=1:  # 只考虑ChatCompletion单文本输入的情况
                raise ValueError(f"chat history is not single text")
            input_content = input[0].get('content','')
            
            # 生成评价模板
            prompt_text, _ = self.evaluation_prompt.render(
                metric_name=self.metric_name,
                criteria=self.prompt_metrics,
                steps=self.prompt_steps,
                max_score=self.prompt_max_score,
                prompt=input_content,
                response=reference,
            )
            
            # 调用模型获得评分
            msg = qianfan.Messages()
            msg.append(prompt_text)
            
            resp = self.model.do(
                messages=msg,
                temperature=0.1,
                top_p=1,
            )
            
            # print(f'{self.metric_name}|{input[0]}|{output}|{resp["result"].strip()}')
            return {self.metric_name: resp["result"].strip()}
        else:
            raise ValueError(f"input in {type(input)} not supported")

In [148]:
import qianfan
from qianfan.model import Service

eb_turbo_service = Service(model="ERNIE-Bot-turbo")  # 加载生成回答的服务
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")  # 实例化用于裁判的模型

[WARNING] [02-01 14:11:26] model.py:375 [t:8602448512]: service type should be specified before exec


附加本地评估器，供EvaluationManager调用

In [149]:
local_evaluators = []
for eval_type, (criteria, steps, max_score) in evaluation_metrics.items():
    local_evaluator = LocalJudgeEvaluator(
        evaluation_prompt=Prompt(evaluation_prompt_template),
        prompt_metrics=criteria,
        prompt_steps=steps,
        prompt_max_score=max_score,
        model=chat_comp,
        metric_name=eval_type
    )
    local_evaluators.append(local_evaluator)

In [150]:
from qianfan.evaluation import EvaluationManager

em = EvaluationManager(local_evaluators=local_evaluators)
result = em.eval(
    [eb_turbo_service], ds,
)

In [151]:
print(result.result_dataset.list())

[{'input_chats': [{'content': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'role': 'user'}], 'expected_output': '修改后的立法法全文公布', 'model_content': [{'Coherence': '评分：7\n\n原因：该摘要简洁明了地传达了新闻的主要内容，即“修改后的立法法全文公布”。然而，摘要没有涵盖新闻中的所有关键信息，例如立法法被分为6章、共计105条等细节。此外，摘要也没有明确提及“新华社受权于18日全文播发”这一重要信息。尽管如此，该摘要仍然以清晰且逻辑的顺序呈现了新闻的主题。因此，我认为该摘要可以得到一个中等的评分。', 'Consistency': '评分: 8\n原因: 摘要“修改后的立法法全文公布”确实反映了新闻的主要内容，即修改后的《中华人民共和国立法法》已经全文播发。然而，摘要省略了一些细节，比如立法法的具体章节和条数。尽管这些细节在某些情况下可能不重要，但它们仍然是原文的一部分。因此，在一致性的评分上，我会稍微扣一点分，以反映摘要并未包含所有原文信息的情况。但总体来说，摘要还是很好地捕捉到了新闻的主旨。', 'Fluency': '评分: 3\n原因: 摘要“修改后的立法法全文公布”非常简洁且流畅，没有语法、拼写或标点错误。它准确地传达了新闻的主要内容，即修改后的立法法已经全文公布。虽然摘要省略了一些细节，如立法法的章节和条数，但这些信息对于传达新闻的主要内容并不是必需的。因此，这个摘要的流畅度得分是3。', 'Relevance': '评分: 8\n原因: 摘要“修改后的立法法全文公布”准确地捕捉到了新闻中的关键信息，即修改后的《中华人民共和国立法法》已经全文发布。虽然摘要没有提到立法法的具体章节和条数，但这部分信息对于一般读者来说可能不是最重要的。摘要避免了冗余和多余信息，简洁明了地传达了新闻的主要内容。因此，我认为这个摘要在相关性方面做得很好，给出了8分的高分。', 'content': '是的，你提到的修改后的《中华人民共和国立法法》已由中华人民共和国第十三届全国人民代表大会第三次会议于2023年3月5

In [155]:
def split_scores(judge_result):
    scores:dict = judge_result['model_content'][0]
    for metric_name, content in scores.items():
        print(metric_name, content)
        if metric_name not in ('content', "llm_tag"):
            scores[metric_name] = content.split('\n')[0].replace('评分: ', '').replace('评分：', '')
    return judge_result

result_dataset = result.result_dataset
result_dataset.map(split_scores)
print(result_dataset.list())

Coherence 评分：7
Consistency 8
Fluency 3
Relevance 8
content 是的，你提到的修改后的《中华人民共和国立法法》已由中华人民共和国第十三届全国人民代表大会第三次会议于2023年3月5日通过并由新华社受权播发。此次修改后的立法法共分六章、一百零五条，其中涵盖了总则、法律、行政法规、地方性法规、自治条例和单行条例、规章、适用与备案审查以及附则等内容。

这次立法法的修改是为了适应新时代全面推进依法治国的新要求，进一步推进科学立法、民主立法、依法立法，提高立法质量和效率，不断完善中国特色社会主义法律体系。修改后的立法法将更好地保障人民通过宪法和法律管理国家事务，管理经济和文化事业，维护自身合法权益，推动全面依法治国实践不断深入。
llm_tag None_None_ERNIE-Bot-turbo
Coherence 6
Consistency 7
Fluency 2
Relevance 8
content 这起事故是非常悲惨的，对于受害者和他们的家人来说，这是一次无法挽回的损失。对于司机和她的家人来说，这也是一次深刻的教训，提醒他们要时刻关注交通安全，遵守交通规则，避免超速行驶和操作不当等危险行为。

深圳市交警局的通报是非常及时的，他们提供了关于事故发生的原因和情况的详细信息。这有助于公众了解事故的经过，也有助于防止类似的事故再次发生。同时，他们也表示了对于伤者的关心和支持，并提供了相关的治疗和赔偿措施。

对于事故赔偿费超过一千万元的情况，这无疑是一笔巨大的开支，对于肇事司机和她的家庭来说也是一个沉重的负担。但是，这也是一个警示，提醒人们在事故发生后要积极承担责任，及时进行赔偿，以尽可能地减少损失和影响。

总之，这起事故再次提醒我们，交通安全是我们每个人都应该关注和重视的问题。我们需要时刻保持警惕，遵守交通规则，避免超速行驶和操作不当等危险行为，以保护自己和他人的安全。
llm_tag None_None_ERNIE-Bot-turbo
Coherence 2
Consistency 2
Fluency 评分：1
Relevance 2
content 关于这个问题，您可以参阅相关内容网站，您也可以问我一些其他问题，我会尽力为您解答。
llm_tag None_None_ERNIE-Bot-turbo
[{'inp

In [156]:
result_df = {"News id":[],"Evaluation Type":[],"Score":[]}
for i, resp in enumerate(result_dataset.list()):
    for metric_name in evaluation_metrics.keys():
        result_df["News id"].append(i)
        # 添加评估类型及对应得分，因为此处只用了一个大模型，所以只取第一个结果
        result_df["Evaluation Type"].append(metric_name)
        result_df["Score"].append(resp["model_content"][0][metric_name])
    print(resp)

{'input_chats': [{'content': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'role': 'user'}], 'expected_output': '修改后的立法法全文公布', 'model_content': [{'Coherence': '7', 'Consistency': '8', 'Fluency': '3', 'Relevance': '8', 'content': '是的，你提到的修改后的《中华人民共和国立法法》已由中华人民共和国第十三届全国人民代表大会第三次会议于2023年3月5日通过并由新华社受权播发。此次修改后的立法法共分六章、一百零五条，其中涵盖了总则、法律、行政法规、地方性法规、自治条例和单行条例、规章、适用与备案审查以及附则等内容。\n\n这次立法法的修改是为了适应新时代全面推进依法治国的新要求，进一步推进科学立法、民主立法、依法立法，提高立法质量和效率，不断完善中国特色社会主义法律体系。修改后的立法法将更好地保障人民通过宪法和法律管理国家事务，管理经济和文化事业，维护自身合法权益，推动全面依法治国实践不断深入。', 'llm_tag': 'None_None_ERNIE-Bot-turbo'}]}
{'input_chats': [{'content': '一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。', 'role': 'user'}], 'expected_output': '深圳机场9死24伤续：司机全责赔偿或超千万', 'model_content': [{'Coherence': '6', 'Consistency': '7', 'Fluency': '2', 'Relevance': '8', 'content': '这起事故是非常悲惨的，对于受害者和他们的家人来说，这是一次无法挽回的损失。对于司机和她的家人来说，这也是一次深刻的教训，提醒他们要时刻关注交通安全，遵守交通规则，避免超速行驶和

In [157]:
import pandas as pd
pivot_df = pd.DataFrame(result_df, index=None).pivot(
    columns="Evaluation Type", index="News id", values="Score"
).astype(int)
pivot_df['Mean_Score'] = pivot_df.mean(axis=1)
display(pivot_df)

Evaluation Type,Coherence,Consistency,Fluency,Relevance,Mean_Score
News id,,,,,
0,7,8,3,8,6.50
1,6,7,2,8,5.75
2,2,2,1,2,1.75


## 方法二：封装评估函数

除了上述进行本地评估的方法，也可以用ChatCompletion模型构造评估函数，从头开始制作评估的流程

In [166]:
import qianfan

def get_geval_score(chat_comp, evaluation_prompt, **kwargs):
    prompt, _ = evaluation_prompt.render(**kwargs)
    msg = qianfan.Messages()
    msg.append(prompt, role='user')
    resp = chat_comp.do(
        messages=msg,
        temperature=0.1,
        top_p=1,
    )
    return resp['result']

def split_scores_str(judge_result):
    return judge_result.split('\n')[0].replace('评分: ', '').replace('评分：', '')

最后遍历数据集进行评估

In [174]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")
result = {"Evaluation Type": [], "News id": [], "Score": []}
for eval_type, (criteria, steps, max_score) in evaluation_metrics.items():
    for ind, data in enumerate(ds.list()):
        result["Evaluation Type"].append(eval_type)
        result["News id"].append(ind)
        evaluate = get_geval_score(
            chat_comp,
            evaluation_prompt,
            criteria=criteria,
            steps=steps,
            prompt=data[0]['prompt'],
            response=data[0]['response'][0][0],
            max_score=str(max_score),
            metric_name=eval_type
        )
        score = int(split_scores_str(evaluate.strip()))
        result["Score"].append(score)


In [175]:
print(result)

{'Evaluation Type': ['Relevance', 'Relevance', 'Relevance', 'Coherence', 'Coherence', 'Coherence', 'Consistency', 'Consistency', 'Consistency', 'Fluency', 'Fluency', 'Fluency'], 'News id': [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2], 'Score': [8, 8, 2, 7, 7, 2, 8, 7, 2, 3, 2, 1]}


查看最终结果

In [176]:
pivot_df = pd.DataFrame(result, index=None).pivot(
    columns="Evaluation Type", index="News id", values="Score"
).astype(int)
pivot_df['Mean_Score'] = pivot_df.mean(axis=1)
display(pivot_df)

Evaluation Type,Coherence,Consistency,Fluency,Relevance,Mean_Score
News id,,,,,
0,7,8,3,8,6.50
1,7,7,2,8,6.00
2,2,2,1,2,1.75
